# Decorators in Python
## By Allen Huang

1. First-class functions
2. Closures
____
*Above the line, it's some prerequisite of decorator*
3. Use function as decorators
4. Use class as decorators
5. Practical examples

### 1. First-class functions

A programming language is said to have First-class functions when functions in that language are treated like any other variable. For example, in such a language, a function can be passed as an argument to other functions, can be returned by another function and can be assigned as a value to a variable.

### 1.1 Assign a function to a variable

In [7]:
def square(x):
    return x*x

In [3]:
f = square(5)
print(square)
print(f)

<function square at 0x105e9b950>
25


In [5]:
# ()means we want to excute the funtions, but we do not.
f = square
print(square)
print(f)
# now, we can treat the variable f as a funtion

<function square at 0x105e9b950>
<function square at 0x105e9b950>


In [6]:
f(5)

25

### 1.2 Passing a function as an argument

In [8]:
def my_map(func, arg_list):
    result = []
    for i in arg_list:
        result.append(func(i))
    return result

In [9]:
squares  = my_map(square, [1,2,3,4,5])
print(squares)

[1, 4, 9, 16, 25]


In [12]:
list(map(square, [1,2,3,4,5]))

[1, 4, 9, 16, 25]

### 1.3 Return a function from another function

In [14]:
def logger(msg):
    # the log_message function do not have any argument
    def log_message():
        print('Log:', msg)
    return log_message
log_hi = logger('Hi')
# we pass in 'Hi', and the function return log_message with excute, and pass this function to log_hi

In [15]:
# run this variable just like a function. In fact, it is a function.
log_hi()

Log: Hi


In [16]:
# example
def html_tag(tag):
# this function take an argument tag
    def wrap_text(msg):
        # print out tag and msg in some format
        print('<{0}>{1}</{0}>'.format(tag, msg))

    return wrap_text
    # returning this function, without excute

In [18]:
print_h1 = html_tag('h1')
print(print_h1)
# so it's waiting to be excute

<function html_tag.<locals>.wrap_text at 0x105ed0268>


In [20]:
print_h1('Test Headline!')
print_h1('Another Headline!')

<h1>Test Headline!</h1>
<h1>Another Headline!</h1>


In [21]:
print_p = html_tag('p')
print_p('Test Paragraph!')

<p>Test Paragraph!</p>


### 2. Closures

Operationally, a closure is a record storing a function together with an environment. The environment is a mapping associating each free variable of the function (variables that are used locally, but defined in an enclosing scope) with the value or reference to which the name was bound when the closure was created.Unlike a plain function, a closure allows the function to access those captured variables through the closure's copies of their values or references, even when the function is invoked outside their scope.

In [23]:
def outer_func():
    # assign the variable 
    message = 'Hi'
    def inner_func():
        # here, message is a free variable, we haven't define it in the inner function, but we still have access it within the inner function
        print(message)
    # excute the inner_func and return it     
    return inner_func()

outer_func()

Hi


In [25]:
def outer_func():
    message = 'Hi'
    
    def inner_func():
        print(message)
    # just return the function
    return inner_func

my_func = outer_func()

In [26]:
print(my_func.__name__)

inner_func


In [27]:
my_func()

Hi


### 3. Use function as decorators

A closure is an inner function that remebers and has access to variable in the local scope in which it was created, even if after the outer function has finished executing.

In [28]:
def outer_func(msg):
    message = msg
    
    def inner_func():
        print(message)

    return inner_func

hi_func = outer_func('Hi')
hello_func = outer_func('Hello')

In [29]:
hi_func()
hello_func()

Hi
Hello


In [32]:
import logging
logging.basicConfig(filename='/Users/hkmac/Desktop/Carzy_Allen_Github/Data_and_Testfile/example.log', level=logging.INFO)


def logger(func):
# this function takes func as it's parameter
    def log_func(*args):
        # taks in any number of arguments 
        logging.info(
            'Running "{}" with arguments {}'.format(func.__name__, args))
        print(func(*args))
    return log_func


def add(x, y):
    return x+y


def sub(x, y):
    return x-y

add_logger = logger(add)
sub_logger = logger(sub)

add_logger(3, 3)
add_logger(4, 5)

sub_logger(10, 5)
sub_logger(20, 10)

6
9
5
10


Decorator: a function that take another function as an argument, return another function. Help us easily add functionality to our existing functions.

In [38]:
def decorator_function(original_function):
    def wrapper_function():
        print(f"warpper executed this before {original_function.__name__}")
        return original_function()
    return wrapper_function
    # return the wrapper_function and waiting to be excuted. When it's beening excuted, print and excute ori function

In [37]:
def display():
    print('display function ran')
    
decorator_display = decorator_function(display)
decorator_display()

warpper executed this before display
display function ran


__@decorator_function__    and   __decorator_display = decorator_function(display)__    is the samething

In [39]:
@decorator_function
def display():
    print('display function ran')
    
display()

warpper executed this before display
display function ran


In [40]:
def display_info(name,age):
    print(f'display_info ran with arguments ({name},{age})')
display_info('Harry', 20)

display_info ran with arguments (Harry,20)


In [42]:
# if we want both display() and display_info() to be added functionality for decorator_function. We need set the wrapper_function's argument

def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print(f"warpper executed this before {original_function.__name__}")
        return original_function(*args, **kwargs)
    return wrapper_function

@decorator_function
def display():
    print('display function ran')

@decorator_function    
def display_info(name,age):
    print(f'display_info ran with arguments ({name},{age})')
    
display_info('Harry', 20)
display()

warpper executed this before display_info
display_info ran with arguments (Harry,20)
warpper executed this before display
display function ran


解析：首先，decorater_function 接受一个original_function 作为实参，inner function也就是wrapper_function, 接受任何数量的positional或者key word arguments，执行print，并返回original_function执行的结果，但是对于outer function，返回待执行的wrapper_function. 对于display()，我们并没有传入任何argument，也就是将display传入outer function。同理，display_info()也被执行。

关于args and kwargs：https://www.jianshu.com/p/0ed914608a2c?utm_source=oschina-app

### 4. Use class as decorators

In [43]:
class decorator_class(object):

    def __init__(self, original_function):
    # self for the instance, and original_function
        # tie our function with the instance of our class 
        self.original_function = original_function
    # behave just like wrapper_function
    def __call__(self, *args, **kwargs):
        print('call method before {}'.format(self.original_function.__name__))
        return self.original_function(*args, **kwargs)

In [44]:
@decorator_class
def display():
    print('display function ran')

@decorator_class   
def display_info(name,age):
    print(f'display_info ran with arguments ({name},{age})')
    
display_info('Harry', 20)
display()

call method before display_info
display_info ran with arguments (Harry,20)
call method before display
display function ran


### 5. Practical examples

__Ex1: How many times a specific functions is run? What arguments are passing to that function?__

In [49]:
# Decorators
from functools import wraps


def my_logger(orig_func):
# passing in an orig_func
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)
    # setting up a log file, that matches the name of our orginal function

    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        # run the orig_func with arguments and keywords, return that result
        return orig_func(*args, **kwargs)

    return wrapper

@my_logger
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Tom', 22)

display_info ran with arguments (Tom, 22)


__Ex2: How long function is run?__

In [48]:
def my_timer(orig_func):
    import time
     
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

import time


@my_timer
def display_info(name, age):
    # take 1 sec to run that function
    time.sleep(1)
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Tom', 22) 

display_info ran with arguments (Tom, 22)
display_info ran in: 1.0016498565673828 sec


__Ex3: Chain some decoraters together!__

In [51]:
def my_logger(orig_func):
# passing in an orig_func
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper

def my_timer(orig_func):
    import time
     
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

import time

@my_logger
@my_timer
def display_info(name, age):
    # take 1 sec to run that function
    time.sleep(1)
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Tom', 22)
print(display_info.__name__)

display_info ran with arguments (Tom, 22)
display_info ran in: 1.0048320293426514 sec
wrapper


In [57]:
from functools import wraps

def my_logger(orig_func):
# passing in an orig_func
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper

def my_timer(orig_func):
    import time
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

@my_logger
@my_timer
def display_info(name, age):
    # take 1 sec to run that function
    time.sleep(1)
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Tom', 22)

display_info ran with arguments (Tom, 22)
display_info ran in: 1.001405954360962 sec
